In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [ ]:
# Data load
tariff_plans = pd.read_csv('Tariff_plans_change.csv')
charges = pd.read_csv('Charges.csv')
suspended = pd.read_csv('Suspended.csv')

# String data to datetime
tariff_plans['START_DTTM'] = pd.to_datetime(tariff_plans['START_DTTM'])
tariff_plans['END_DTTM'] = pd.to_datetime(tariff_plans['END_DTTM'], errors='coerce')
charges['BILL_MONTH'] = pd.to_datetime(charges['BILL_MONTH'])
suspended['START_DT'] = pd.to_datetime(suspended['START_DT'])
suspended['END_DT'] = pd.to_datetime(suspended['END_DT'], errors='coerce')

In [3]:
tariff_plans.head(10)

,SUBSCRIBER_ID,TARIFF_PLAN_ID,START_DTTM,END_DTTM
0,1,2,2013-11-22 21:10:08,2017-03-11 14:24:17
1,1,3,2017-03-11 14:24:18,NaT
2,2,1,2016-02-05 16:36:34,2017-06-29 12:39:58
3,2,5,2017-06-29 12:39:59,NaT
4,3,1,2015-05-06 19:17:46,2017-04-14 12:00:59
5,3,5,2017-04-14 12:01:00,NaT
6,4,3,2016-01-27 23:03:51,2017-01-09 19:27:50
7,4,5,2017-01-09 19:27:51,NaT
8,5,1,2016-02-06 08:07:15,2017-05-08 12:39:59
9,5,5,2017-05-08 12:40:00,NaT


In [53]:
tariff_plans['SUBSCRIBER_ID'].nunique()

5989

In [54]:
tariff_plans.shape

(12344, 4)

In [4]:
charges.head(10)

,SUBSCRIBER_ID,BILL_MONTH,CHARGES
0,4741,2017-03-01,4.572
1,1754,2017-05-01,4.894
2,381,2017-01-01,6.670
3,5832,2016-12-01,4.299
4,5277,2017-03-01,1.610
5,4512,2017-02-01,3.220
6,1688,2017-05-01,7.391
7,3204,2017-10-01,2.456
8,1277,2016-08-01,11.242
9,5424,2017-04-01,26.986


In [5]:
suspended.head(10)

,SUBSCRIBER_ID,START_DT,END_DT,STATUS
0,5195,2017-12-27,2017-12-27,Suspended
1,5198,2017-04-18,2017-04-18,Suspended
2,381,2017-04-18,2017-04-20,Suspended
3,3500,2017-12-21,NaT,Suspended
4,3302,2016-08-23,2016-09-07,Suspended
5,4512,2017-04-14,2017-04-28,Suspended
6,1738,2016-08-11,2017-01-10,Suspended
7,2294,2017-01-27,2017-01-27,Suspended
8,1277,2017-01-27,2017-03-21,Suspended
9,5424,2016-11-18,2016-11-29,Suspended


## Tariff migration

In [9]:
# Creating dataset for all of migrations

tariff_migrations = tariff_plans.copy()
tariff_migrations['MIGRATION_TARIFF'] = tariff_migrations.groupby('SUBSCRIBER_ID')['TARIFF_PLAN_ID'].shift(-1)
migrations = tariff_migrations.dropna(subset=['MIGRATION_TARIFF']).reset_index()
migration_flows = migrations.groupby(['TARIFF_PLAN_ID', 'MIGRATION_TARIFF']).size().reset_index(name='COUNT')
migration_flows['MIGRATION_TARIFF'] = migration_flows['MIGRATION_TARIFF'].astype('int64')

# Drop rows with same previous and migration tariffs
drop_mask = (migration_flows['TARIFF_PLAN_ID'] == migration_flows['MIGRATION_TARIFF'])
migration_flows.drop(migration_flows[drop_mask].index, inplace=True)
migration_flows

,TARIFF_PLAN_ID,MIGRATION_TARIFF,COUNT
1,1,2,5
2,1,3,55
3,1,4,378
4,1,5,1613
5,2,1,13
6,2,3,20
7,2,4,158
8,2,5,417
9,3,1,28
10,3,2,12


In [52]:
# Creating Sankey diagram for migration flows

labels = (pd.concat([migration_flows['TARIFF_PLAN_ID'], migration_flows['MIGRATION_TARIFF']])
          .unique())
unique_labels_amount = len(labels)
labels = list(map(lambda l: f'Tariff {l}', labels))
sankey_labels = labels * 2

source = migration_flows['TARIFF_PLAN_ID'].apply(lambda x: sankey_labels.index(f'Tariff {x}')).to_list()
target = migration_flows['MIGRATION_TARIFF'].apply(lambda x: sankey_labels.index(f'Tariff {x}', unique_labels_amount)).to_list()
value = migration_flows['COUNT']

fig = go.Figure()
fig.add_trace(
    go.Sankey(
        valuesuffix=' Migrations',
        arrangement='snap',
        node=dict(
            label=sankey_labels,
            pad=20,
            thickness=30,  
            
        ),
        link=dict(
            source=source,
            target=target,
            value=value
        )
    )
)
fig.update_layout(title='Tariff-to-Tariff migration flows diagram. Left side is emigration tariff. Right side is immigration tariff',
                  height=1080)
fig.show()

Most popular emigration tariff: Tariff 3 (~2.78k migrations)

Least popular emigration tariff: Tariff 5 (209 migrations)

Most popular imigration tariff: Tariff 5 (~4.93k migrations)

Least popular imigration tariff: Tariff 2 (41 migrations)

## Average month charge change

In [75]:
avg_charges_df = tariff_plans.copy()

avg_charges_df['CHANGE_MONTH'] = pd.to_datetime(avg_charges_df['START_DTTM']).dt.to_period('M')
avg_charges_df['END_MONTH'] = pd.to_datetime(avg_charges_df['END_DTTM']).dt.to_period('M')
avg_charges_df = avg_charges_df.sort_values(['SUBSCRIBER_ID', 'CHANGE_MONTH'])

avg_charges_df['MIGRATION'] = avg_charges_df.groupby('SUBSCRIBER_ID')['TARIFF_PLAN_ID'].shift(-1)
tariff_migrations = avg_charges_df.dropna(subset=['MIGRATION']).reset_index(drop=True)

tariff_migrations = tariff_migrations[tariff_migrations['TARIFF_PLAN_ID'] != tariff_migrations['MIGRATION']]

charges = charges.sort_values(['SUBSCRIBER_ID', 'BILL_MONTH'])

merged_data = tariff_migrations.merge(charges, on='SUBSCRIBER_ID', how='inner')

def calculate_periods(change_month):
    pre_period = [change_month - i for i in range(1, 4)]
    post_period = [change_month + i for i in range(1, 4)]
    return pre_period, post_period

results = []
for _, row in tariff_migrations.iterrows():
    pre_period, post_period = calculate_periods(row['CHANGE_MONTH'])
    subscriber_id = row['SUBSCRIBER_ID']
    migration = (row['TARIFF_PLAN_ID'], row['MIGRATION'])

    pre_avg = charges[(charges['SUBSCRIBER_ID'] == subscriber_id) & (charges['BILL_MONTH'].isin(pre_period))]['CHARGES'].mean()
    post_avg = charges[(charges['SUBSCRIBER_ID'] == subscriber_id) & (charges['BILL_MONTH'].isin(post_period))]['CHARGES'].mean()

    results.append({
        'TARIFF_PLAN_ID': migration[0],
        'MIGRATION': migration[1],
        'PRE_AVG': pre_avg,
        'POST_AVG': post_avg,
        'DELTA': post_avg - pre_avg
    })

delta_df = pd.DataFrame(results)
aggregated = delta_df.groupby(['TARIFF_PLAN_ID', 'MIGRATION']).agg({
    'DELTA': 'mean',
    'PRE_AVG': 'count'
}).reset_index().rename(columns={'PRE_AVG': 'COUNT'})

aggregated = aggregated[aggregated['COUNT'] > 0]

fig = px.bar(
    aggregated,
    x='DELTA',
    y=aggregated.apply(lambda x: f"Tariff {int(x['TARIFF_PLAN_ID'])} → Tariff {int(x['MIGRATION'])}", axis=1),
    orientation='h',
    color='DELTA',
    color_continuous_scale=['red', 'yellow', 'green'],
    title='Average charge change',
    labels={'DELTA': 'Charge change', 'y': 'Migration direction'}
)
fig.update_layout(xaxis_title='Charge change', yaxis_title='Migration direction',
                  height=1080)
fig.show()

## Suspensions analysis

In [87]:
suspended['START_DT'] = pd.to_datetime(suspended['START_DT'])
suspended['END_DT'] = pd.to_datetime(suspended['END_DT']).fillna(pd.Timestamp('today'))
suspended['START_MONTH'] = suspended['START_DT'].dt.to_period('M')
suspended['END_MONTH'] = suspended['END_DT'].dt.to_period('M')

avg_block_df = tariff_plans.copy()
avg_block_df['CHANGE_MONTH'] = pd.to_datetime(avg_block_df['START_DTTM']).dt.to_period('M')
avg_block_df = avg_block_df.sort_values(['SUBSCRIBER_ID', 'CHANGE_MONTH'])
avg_block_df['MIGRATION'] = avg_block_df.groupby('SUBSCRIBER_ID')['TARIFF_PLAN_ID'].shift(-1)

avg_block_df = avg_block_df[avg_block_df['TARIFF_PLAN_ID'] != avg_block_df['MIGRATION']].dropna(subset=['MIGRATION'])

def calculate_suspensions(subscriber_id, periods):
    count = suspended[
        (suspended['SUBSCRIBER_ID'] == subscriber_id) &
        (suspended['START_MONTH'] <= periods[-1]) & 
        (suspended['END_MONTH'] >= periods[0])
    ].shape[0]
    return count

results = []
for _, row in avg_block_df.iterrows():
    pre_period, post_period = calculate_periods(row['CHANGE_MONTH'])
    subscriber_id = row['SUBSCRIBER_ID']
    migration = (row['TARIFF_PLAN_ID'], row['MIGRATION'])

    pre_blocks = calculate_suspensions(subscriber_id, pre_period)
    post_blocks = calculate_suspensions(subscriber_id, post_period)

    results.append({
        'TARIFF_PLAN_ID': migration[0],
        'MIGRATION': migration[1],
        'PRE_BLOCKS': pre_blocks,
        'POST_BLOCKS': post_blocks,
        'DELTA_BLOCKS': post_blocks - pre_blocks
    })

block_df = pd.DataFrame(results)
aggregated_blocks = block_df.groupby(['TARIFF_PLAN_ID', 'MIGRATION']).agg({
    'PRE_BLOCKS': 'sum',
    'POST_BLOCKS': 'sum',
}).reset_index()

aggregated_blocks['DELTA_BLOCKS'] = aggregated_blocks['POST_BLOCKS'] - aggregated_blocks['PRE_BLOCKS']

fig = px.bar(
    aggregated_blocks,
    x='DELTA_BLOCKS',
    y=aggregated_blocks.apply(lambda x: f"Tariff {int(x['TARIFF_PLAN_ID'])} → Tariff {int(x['MIGRATION'])}", axis=1),
    orientation='h',
    color='DELTA_BLOCKS',
    color_continuous_scale=['green', 'yellow', 'red'],
    title='Change in Suspension Levels After Tariff Migration (Absolute Numbers)',
    labels={'DELTA_BLOCKS': 'Difference of Suspensions amount After Migration', 'y': 'Migration Direction'}
)
fig.update_layout(
    xaxis_title='Difference of Suspensions amount After Migration', 
    yaxis_title='Migration Direction',
    height=1080
)
fig.show()